In [0]:
!git clone https://github.com/acmilannesta/Adult_readmission

In [0]:
!pip install tsfresh
!pip install pandas==0.22

In [0]:
import pandas as pd
from tsfresh import extract_relevant_features, extract_features

In [4]:
lab = pd.read_csv('Adult_readmission/PA_0501.csv')
lab['SPECIMEN_COLLECT_DT'] = pd.to_datetime(lab['SPECIMEN_COLLECT_DT'])
y = pd.read_sas('Adult_readmission/combined_vars_04092019.sas7bdat')[['PERSON_ID', 'CODE_REHOSP']]

(116036, 28)

**Biomarker list created**

"markerlist": biomarkers with multiple time points

'singlemarker': biomarkers with the last time point taken before transplantation

In [0]:
markerlist = ['CREATININE',
           'GLUCOSE',
           'MAGNESIUM',
           'CHLORIDE',
           'ANION_GAP__BLOOD',
           'ALKALINE_PHOSPHATASE',
           'ALBUMIN',
           'BUN__BLOOD_UREA_NITROGEN_',
           'CO2',
           'HEMATOCRIT',
           'HEMOGLOBIN',
           'POTASSIUM_PLASMA',
           'SODIUM_PLASMA',
           'UREA_CREATININE_RATIO',
             ]
singlemarker = ['DRUG_SCREEN', 
                'GLUCOSE_URINE_QUALITATIVE',
                'NITRITE_URINE_QUALITATIVE', 
                'PROTEIN_URINE_QUALITATIVE',
                'HBA1C__HEMOGLOBIN_A1C_',
                'INDIRECT_BILIRUBIN',
                'NON_HDL_CHOLESTEROL',
                'PROGRAF__TACROLIMUS_',
                'PT_RESULT',
                'URIC_ACID',
               ]

In [0]:
biomarker_extract = y['PERSON_ID'].to_frame()
for marker in markerlist:
  biomarker = lab[['PERSON_ID', 'SPECIMEN_COLLECT_DT', marker]].dropna().sort_values(['PERSON_ID', 'SPECIMEN_COLLECT_DT'])
  biomarker['time'] = biomarker.groupby('PERSON_ID').cumcount()
  y_biomarker = biomarker['PERSON_ID'].drop_duplicates().to_frame().merge(y, on='PERSON_ID', how='left')
  y_biomarker.index=y_biomarker['PERSON_ID']  
  f = extract_relevant_features(biomarker.drop('SPECIMEN_COLLECT_DT', 1), y_biomarker['CODE_REHOSP'], column_id='PERSON_ID', column_sort='time')
  f['PERSON_ID'] = f.index
  biomarker_extract = biomarker_extract.merge(f, on='PERSON_ID', how='left')

In [0]:
for marker in singlemarker:
    tmp = lab[['PERSON_ID', marker]].dropna()
    assert(tmp.drop_duplicates().shape[0] == tmp.shape[0])
    biomarker_extract = biomarker_extract.merge(tmp, on='PERSON_ID', how='left')

In [0]:
biomarker_extract.to_csv('Adult_readmission/feature_extracted.csv', index=False)